In [1]:
import sys
sys.path.append('../lib/tfmodels/research')
sys.path.append('../')

In [8]:
import os
import tensorflow as tf
os.chdir("/home/dtch009/tf-detect")

MODEL_NAME = "ssd_mobilenet_v2_96x96_crowdhuman_debug_depth=0.1"

model_savedir = f'exported_models/{MODEL_NAME}/saved_model'
model = tf.saved_model.load(model_savedir)
print(model.__dict__)

def count_parameters(model):
    total_parameters = 0
    for variable in model.variables:
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        total_parameters += variable_parameters
    return total_parameters

# print(f"Model has {count_parameters(model)} parameters")

{'_self_setattr_tracking': True, '_self_unconditional_checkpoint_dependencies': [TrackableReference(name='_model', ref=<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7f6890344a90>), TrackableReference(name='signatures', ref=_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(input_tensor) at 0x7F680E0A50D0>})), TrackableReference(name='_self_saveable_object_factories', ref=DictWrapper({}))], '_self_unconditional_dependency_names': {'_model': <tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7f6890344a90>, 'signatures': _SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(input_tensor) at 0x7F680E0A50D0>}), '_self_saveable_object_factories': DictWrapper({})}, '_self_unconditional_deferred_dependencies': {}, '_self_update_uid': 188394, '_self_name_based_restores': set(), '_self_saveable_object_factories': DictWrapper({}), '_model': <tensorflow.py

In [9]:
converter = tf.lite.TFLiteConverter.from_saved_model(model_savedir, signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
quantized_model = converter.convert()
open(f'exported_models/{MODEL_NAME}/model.tflite', "wb").write(quantized_model)

293216